In [ ]:
from closestRecipe import ClosestRecipe
from farm_vis import farm_vis
import pandas as pd

In [ ]:
cr = ClosestRecipe()

In [ ]:
#this cell demonstrates output of sorting all recipes by distance as a function of an input zip code;
#changing the zip code will alter resulting dataframe
zip_code = '30082'
ranked_recipe_df = cr.recipe_rank_zip(zip_code)

In [ ]:
#this cell sets a lat/ lon position from zip code center &
#demonstrates calculating the mean distance for all recipes from a given input location
lat_long = cr.zip_lookup_lat_long(zip_code)
print('the center of that zip code is at: ' + str(lat_long))
average_recipe_distance_by_lat_long = cr.recipe_rank_avg_lat_long(lat_long)[0][0]
print('the average distance of all recipes from that input location is ' + str(average_recipe_distance_by_lat_long) + ' miles.')

In [ ]:
ranked_with_ingredients = cr.recipe_rank_ings_zip(zip_code)
#ranked_with_ingredients.head()

In [ ]:
pd.merge(cr.recipe_rank_zip('30082'), cr.recipe_rank_ings_zip('30082'), on='uid_recipe_title')

In [ ]:
#this cell generates a chloropleth for average distance of all recipes from all possible county centers in the 
#contiguous United States
#this function takes time to run (approx. 30 minutes locally on laptop)
farm_vis.chloropleth()

In [ ]:

#the cell generates a cumulative density function of recipe distances from a given user input location
import seaborn as sns
import matplotlib.pyplot as plt
def dist_plot(x,y):
    #x = recipe_frame
    #y = current_index
    density = (y+1)/len(x)
    mcdf = sns.kdeplot(x['mean_score'], cumulative=True, shade=True, color="b").set(
        xlabel='recipe average distance', ylabel='% recipes closer')
    plt.plot([y, y], [0, density])